# 뉴스 데이터 수집 프로젝트

# 0. 라이브러리 불러오기

In [24]:
# HTTP 요청 및 HTML 파싱
import requests
from bs4 import BeautifulSoup

#데이터베이스 설계
from sqlalchemy import create_engine, text

# 데이터 및 시간 관리
import pandas as pd
import time
import random
from datetime import datetime
from dateutil import parser

# 예외 처리
from requests.exceptions import RequestException

# MySQL 연동
import pymysql

# 시각화 
from konlpy.tag import Okt  # 한글 명사 추출용
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import re
from collections import Counter

## 1. 수집 대상 뉴스 사이트 선정

매일경제 RSS서비스 경제뉴스 

In [4]:
url = 'https://www.mk.co.kr/rss/30100041/'

## 2. 데이터 구조, MySQL 테이블 설계

In [5]:
# MySQL 기본 접속 정보 입력
user = 'root'
password = '1234'
host = 'localhost'

# 1. 데이터베이스(news_db) 생성
# MySQL 연결
conn = pymysql.connect(
    host=host, user=user, password=password, db='mysql', charset='utf8mb4'
)
conn.autocommit(True)
with conn.cursor() as cursor:
    cursor.execute("CREATE DATABASE IF NOT EXISTS news_db CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
conn.close()

# 2. news_db에 sqlalchemy로 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/news_db?charset=utf8mb4")

# 3. 테이블 생성 쿼리
create_table_sql = """
CREATE TABLE IF NOT EXISTS mk_economy_news (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    link VARCHAR(500) NOT NULL UNIQUE,
    description TEXT,
    pub_date DATETIME,
    image_url VARCHAR(500),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    print('테이블 자동생성 완료')


테이블 자동생성 완료


## 3. 파싱 설계 및 구현

In [10]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'xml')  

news_list = []
for item in soup.find_all('item'):
    title = item.title.get_text(strip=True)
    link = item.link.get_text(strip=True)
    description = item.description.get_text(strip=True)
    pub_date_raw = item.pubDate.get_text(strip=True)
    # 등록일 형태 변환 
    pub_date = parser.parse(pub_date_raw)
    media = item.find('media:content')
    image_url = media['url'] if media and media.has_attr('url') else None

    news_list.append({
        'title': title,
        'link': link,
        'description': description,
        'pub_date': pub_date,
        'image_url': image_url
    })

# 결과 확인
for news in news_list[:3]: 
    print(news)


{'title': '대출 막힌 취약층…휴대전화 소액결제까지 손 벌린다', 'link': 'https://www.mk.co.kr/news/economy/11379878', 'description': '제2금융권 등 대출처 줄어 불법업체 수수료에 서민 울상신용대출 한도를 연봉 이내로 제한한 ‘6·27 가계부채 대책’ 이후 급전이 필요한 취약계층이 휴대전화 소액결제까지 손대고 있다..', 'pub_date': datetime.datetime(2025, 7, 29, 10, 23, 58, tzinfo=tzoffset(None, 32400)), 'image_url': 'https://pimg.mk.co.kr/news/cms/202507/29/news-p.v1.20250729.6122a58ff1e0429aa415a93a92922c63_R.png'}
{'title': '“저축은행도 AI 활용합니다”…챗GPT ‘열공’한다', 'link': 'https://www.mk.co.kr/news/economy/11379875', 'description': 'JT저축은행, 한국폴리텍대학과 교육 진행  데이터 분석 교육에 임직원 50여명 참여JT저축은행이 한국폴리텍대학과 인공지능 시대 임직원 디지털 역량 강화를 위한 챗GPT 활용 교육을..', 'pub_date': datetime.datetime(2025, 7, 29, 10, 22, 3, tzinfo=tzoffset(None, 32400)), 'image_url': 'https://pimg.mk.co.kr/news/cms/202507/29/news-p.v1.20250729.07718469886c432db23f6bf85a2a9025_R.jpeg'}
{'title': '시중은행, 전체 대출 늘리는데...자영업자만 줄었다', 'link': 'https://www.mk.co.kr/news/economy/11379817', 'description': '가계·기업 대출 40조 증가했지만 상반기 소상공인 여신은 4조 감소올 상반기 주요 시중은행이 가

## 4. 데이터 수집, 정합성 검증, SQL DB연결 파이프라인 개발 

In [15]:
# DB 접속 정보
user = 'root'
password = '1234'
host = 'localhost'
db = 'news_db'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4")

# SQL INSERT 쿼리
insert_sql = """
    INSERT INTO mk_economy_news
    (title, link, description, pub_date, image_url)
    VALUES (:title, :link, :description, :pub_date, :image_url)
"""

# 필수값 정합성 체크 함수
def is_valid(news_item):
    required_fields = ['title', 'link', 'pub_date']
    return all(news_item.get(field) for field in required_fields)

# 뉴스 DB 저장 함수
def insert_news(news_item):
    if not is_valid(news_item):
        print(f"[정합성 오류] {news_item}")
        return

    with engine.begin() as conn: 
        # 중복 체크 
        result = conn.execute(
            text("SELECT COUNT(*) FROM mk_economy_news WHERE link = :link"),
            {'link': news_item['link']}
        )
        count = result.scalar()
        if count == 0:
            conn.execute(text(insert_sql), news_item)
            print(f"[저장됨] {news_item['title']}")
        else:
            print(f"[중복 스킵] {news_item['title']}")

# 전체 뉴스 저장 
for news in news_list:
    insert_news(news)


[중복 스킵] 대출 막힌 취약층…휴대전화 소액결제까지 손 벌린다
[중복 스킵] “저축은행도 AI 활용합니다”…챗GPT ‘열공’한다
[중복 스킵] 시중은행, 전체 대출 늘리는데...자영업자만 줄었다
[중복 스킵] 회장 이어 행장도…신한은행, 우수고객 간담회 개최
[중복 스킵] 고르기 어려운 중고차…인증중고차 플랫폼서 전용 할부상품 활용할만
[중복 스킵] 말로만 친기업 외친 여당…관세전쟁 와중에 ‘반기업 3법’ 속도
[중복 스킵] 매경이 전하는 세상의 지식 (매-세-지, 7월 29일)
[중복 스킵] 요즘 은행가면 예금대신 여기에 돈 넣는다는데…상반기 5조 몰린 이 상품
[중복 스킵] 유튜브 영향력 대단하네…대한민국 잠자는 시간 처음으로 줄었다
[중복 스킵] “카드값 못 막는 사람 너무 많아”…상반기 카드사 실적 타격, 순익 18% 감소
[중복 스킵] “해외에서 K쇼핑하고 싶어도 못해요”…이것 하나 때문에 꽉 막힌 역직구
[중복 스킵] “AI 투자한다고 당장 수익 못거둬, 손실 견뎌야 큰 것 얻어” 입모은 경영구루들
[중복 스킵] “너도 샀어? 나도 그걸로 뽑았어”…요즘 신차 10대 중 1대는 전기차
[중복 스킵] AI로 진화한 보이스피싱, AI로 막는다…의심계좌 포착되면 즉시 차단도
[중복 스킵] 상호관세 인하, 15%가 마지노선…대미 투자 즉석에서 늘리는 ‘트럼프의 협상’
[중복 스킵] 이재명 대통령 ‘이자놀이’ 경고에…은행들 부동산대출 벽 높인다
[중복 스킵] 조선업 협력 제안하며 미국 막판 설득…한미 관세협상 쌀·소고기도 협상 테이블에
[중복 스킵] 금융위 부위원장 지인도 당했다는데…“피싱 막으려면, 주소록에 어머니 저장 금물”
[중복 스킵] 국정위 "첨단산업펀드 150조+α 증액 검토"
[중복 스킵] “소송 이겨봤자 결국 빈손”...대기업한테 기술 뺏긴 중기 6곳중 1곳 망했다
[중복 스킵] 금융·통신·수사 데이터 싹다 모아 사전 탐지…보이스피싱 국가 컨트롤타워 만든다
[중복 스킵] 일본도 EU도 상호관세 15% 도장 ‘꽝’...핀치 몰린 한국의 전략은
[

## 5. 품질점검 요약 리포트 

In [26]:
# DB 정보 
user = 'root'
password = '1234'
host = 'localhost'
db = 'news_db'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4")

# 1. 전체 뉴스 기사 개수 집계
with engine.connect() as conn:
    total_count = conn.execute(text("SELECT COUNT(*) FROM mk_economy_news")).scalar()
print(f"\n### 전체 뉴스 기사 개수: {total_count}건\n")

# 2. 최신 기사 10건
query = """
SELECT title, description, pub_date, link
FROM mk_economy_news
ORDER BY pub_date DESC
LIMIT 10
"""
df = pd.read_sql(query, engine)

print("### 최신 뉴스 10건 요약\n")
for idx, row in df.iterrows():
    print(f"{idx+1}. [{row['pub_date']}] {row['title']}")
    print(f"   - 요약: {row['description'][:100]}...")  # 요약문 100자만
    print(f"   - 링크: {row['link']}\n")

# 3. 크롤링 성공/실패 로그 요약 )
log_path = 'log.txt'  
try:
    with open(log_path, encoding='utf-8') as f:
        log_lines = f.readlines()
    # 실패 로그만 출력 예시
    fail_logs = [line for line in log_lines if '[실패]' in line]
    print(f"\n### 크롤링 실패 로그 요약 ({len(fail_logs)}건)\n")
    for line in fail_logs[:5]:  # 상위 5개
        print(line.strip())
except FileNotFoundError:
    print("\n(별도 실패 로그 파일이 없어, 실패 기록은 생략합니다.)")

# 크롤링에 성공한 뉴스 수(=DB insert 성공 수)는 위 전체 기사 개수와 같습니다.
print("\n=== 품질 점검 리포트 생성 완료 ===\n")



### 전체 뉴스 기사 개수: 50건

### 최신 뉴스 10건 요약

1. [2025-07-29 10:23:58] 대출 막힌 취약층…휴대전화 소액결제까지 손 벌린다
   - 요약: 제2금융권 등 대출처 줄어 불법업체 수수료에 서민 울상신용대출 한도를 연봉 이내로 제한한 ‘6·27 가계부채 대책’ 이후 급전이 필요한 취약계층이 휴대전화 소액결제까지 손대고 있다...
   - 링크: https://www.mk.co.kr/news/economy/11379878

2. [2025-07-29 10:22:03] “저축은행도 AI 활용합니다”…챗GPT ‘열공’한다
   - 요약: JT저축은행, 한국폴리텍대학과 교육 진행  데이터 분석 교육에 임직원 50여명 참여JT저축은행이 한국폴리텍대학과 인공지능 시대 임직원 디지털 역량 강화를 위한 챗GPT 활용 교육을...
   - 링크: https://www.mk.co.kr/news/economy/11379875

3. [2025-07-29 09:46:19] 시중은행, 전체 대출 늘리는데...자영업자만 줄었다
   - 요약: 가계·기업 대출 40조 증가했지만 상반기 소상공인 여신은 4조 감소올 상반기 주요 시중은행이 가계·기업 대출을 늘린 가운데 유독 자영업자 등 소호 대출만 감소한 것으로 나타났다. ...
   - 링크: https://www.mk.co.kr/news/economy/11379817

4. [2025-07-29 09:20:37] 회장 이어 행장도…신한은행, 우수고객 간담회 개최
   - 요약: 진옥동 신한금융 회장 이어 정상혁 행장도 파워유저 초청신한은행이 지난 28일 정상혁 신한은행장 주최로 서울 중구 소재 본점에서 고객 초청 간담회 ‘현장을 듣다, 실행으로 답하다’를...
   - 링크: https://www.mk.co.kr/news/economy/11379784

5. [2025-07-29 08:35:56] 고르기 어려운 중고차…인증중고차 플랫폼서 전용 할부상품 활용할만
   - 요약: 현대자동차 인증 중고

# 6. 결론

매일 경제부문 뉴스의 데이터를 RSS 파싱을 통해 추출,<br>
MySQL DB에 정합성, 중복관리기능의 안정적인 파이프라인을 구축 성공<br>
데이터 품질결과 요약 리포트를 통해 모두 정상적으로 저장, 관리되고 있음을 확인<br>
경제 뉴스 데이터 수집 프로젝트를 달성했다고 판단